<a href="https://colab.research.google.com/github/m0hit-kumar/RASA/blob/main/BERT_for_TASC.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!unzip Corrected_data.zip


Archive:  Corrected_data.zip
   creating: Corrected_data/
  inflating: Corrected_data/BombBlast(categorised).json  
  inflating: Corrected_data/Earthquake(categorised).json  
  inflating: Corrected_data/flood(categorised).json  
  inflating: Corrected_data/tornado(categorised).json  
  inflating: Corrected_data/tsnuami (categorised).json  
  inflating: Corrected_data/Volcano(categorised).json  
  inflating: Corrected_data/war(categorised).json  


In [ ]:
import json
import os

all_data = []
for filename in os.listdir('Corrected_data/'):
    if filename.endswith(".json"):
        file_path = os.path.join('Corrected_data/', filename)
        with open(file_path, 'r') as file:
            data = json.load(file)
            all_data.extend(data)

with open("Data.json", 'w') as outfile:
    json.dump(all_data, outfile, indent=4)

In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
import nltk
nltk.download('stopwords')
import copy
import matplotlib.pyplot as plt
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.preprocessing import LabelEncoder
from sklearn.feature_selection import VarianceThreshold
from imblearn.over_sampling import SMOTE
from sklearn.dummy import DummyClassifier
from sklearn.naive_bayes import MultinomialNB
from sklearn.tree import DecisionTreeClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report
import seaborn as sns
from sklearn.metrics import f1_score


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [ ]:

data=pd.read_json("Data.json")

In [ ]:
data.head()

,text,intent
0,The officers then violently yanked Ms. Young a...,Miscellaneous
1,"On Tuesday, it was announced that the ""Full Ho...",Miscellaneous
2,‘We are walking a very tight line between tack...,Miscellaneous
3,Warning: Graphic content Death threat to Lenno...,Micscellaneous
4,JUST WATCHEDObama warns the foundation of demo...,Micsellaneous


In [ ]:
data["intent"].unique()

array(['Miscellaneous', 'Micscellaneous', 'Micsellaneous', 'War',
       'Micellaneous', 'miscellaneous', 'Miscellaeous', 'Volcano',
       'Tornado', 'Miscellanous', 'Miscellenous', 'Miscelleneous',
       'BombBlast', 'Earthquake', nan, 'floods', 'Miscellaenous',
       'Miscellneous'], dtype=object)

In [ ]:
# checlking for nan value
data['intent'].fillna('misc', inplace=True)

data.loc[data["intent"].str.startswith("m"),"intent"]="misc"
data.loc[data["intent"].str.startswith("M"),"intent"]="misc"

In [ ]:
data["intent"].unique()

array(['misc', 'War', 'Volcano', 'Tornado', 'BombBlast', 'Earthquake',
       'floods'], dtype=object)

In [ ]:
import nltk
nltk.download('stopwords') # for removing useless data
nltk.download('punkt') # use unsupervised algorithm to build a model for abbreviation words, collocations, and words
nltk.download('wordnet') #used  in information systems, word-sense disambiguation, information retrieval, automatic text classification, summarization, machine translation.
nltk.download('omw-1.4')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...


True

In [ ]:
# clean the function will apply the above mentioned preprocessing to the resumes
def clean(text):
    wn = nltk.WordNetLemmatizer()
    stopword = nltk.corpus.stopwords.words('english')
    tokens = nltk.word_tokenize(text)
    lower = [word.lower() for word in tokens]
    no_stopwords = [word for word in lower if word not in stopword]
    no_alpha = [word for word in no_stopwords if word.isalpha()]
    lemm_text = [wn.lemmatize(word) for word in no_alpha]
    clean_text = lemm_text
    # make list of text into string
    result=" ".join(clean_text)
    return result

In [ ]:
!pip install transformers
!pip install torch


In [ ]:
# from transformers import BertTokenizer, TFBertModel
import numpy as np
from transformers import AutoTokenizer, AutoModel
import torch
import tensorflow as tf

In [ ]:
model_name="bert-base-uncased"
tokenizer = AutoTokenizer.from_pretrained(model_name) #hugging face bert tokenizer
model = AutoModel.from_pretrained(model_name) #hugging face bert model for embedings


In [ ]:
def token_and_vector(features, batch_size=16):
    clean_texts = [clean(text) for text in features]
    vectors = []
    for i in range(0, len(clean_texts), batch_size):
        batch = clean_texts[i:i+batch_size]
        texts_tokens= tokenizer.batch_encode_plus(batch, padding=True, truncation=True, return_tensors='pt', max_length=512)
        with torch.no_grad():
            outputs = model(**texts_tokens)
            vectors_batch = outputs.last_hidden_state.mean(dim=1).numpy()
        vectors.append(vectors_batch)
    return np.concatenate(vectors, axis=0)



In [ ]:
features = token_and_vector(data["text"])

In [ ]:
x_train, x_test, y_train, y_test = train_test_split(features,data["intent"],test_size=0.2, random_state=42)
len(x_train),len(x_test)

(390, 98)

In [ ]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.naive_bayes import MultinomialNB
from sklearn.svm import SVC
from sklearn.model_selection import cross_val_score
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()

rfc = RandomForestClassifier()   # Random Forest
mbc = MultinomialNB(force_alpha=True) # MultinomialNB
svc = SVC(kernel='linear', C=1.0) # Support Vector Machine
lrc=LogisticRegression() # Logistic regression




rfc.fit(x_train,y_train)
mbc.fit(scaler.fit_transform(x_train),y_train)
svc.fit(x_train,y_train)
lrc.fit(x_train,y_train)

# traning using the 70% data we have extracted from the dataset
# scores = cross_val_score(rf,X_train,y_train,cv=5)

/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


LogisticRegression()

In [ ]:
models=[ rfc,mbc,svc,lrc]
models_name=["Random Forest","MultinomialNB","Support Vector Machine","Logistic regression"]
models_list={}
for i,j in zip(models_name,models):
  models_list[i]=j
  print(i,j)


Random Forest RandomForestClassifier()
MultinomialNB MultinomialNB(force_alpha=True)
Support Vector Machine SVC(kernel='linear')
Logistic regression LogisticRegression()


In [ ]:
models_list

{'Random Forest': RandomForestClassifier(),
 'MultinomialNB': MultinomialNB(force_alpha=True),
 'Support Vector Machine': SVC(kernel='linear'),
 'Logistic regression': LogisticRegression()}

In [ ]:
from sklearn.metrics import f1_score

def accuracy_score(models,X_test,y_test):
  f1_scores={}
  for model in models.keys():
    pred=models[model].predict(X_test)
    score=f1_score(y_test,pred,average='micro')
    f1_scores[model]=score
  return f1_scores



In [ ]:
f1_scores=accuracy_score(models_list,x_test,y_test)
print(f1_scores)


{'Random Forest': 0.7244897959183674, 'MultinomialNB': 0.6224489795918368, 'Support Vector Machine': 0.806122448979592, 'Logistic regression': 0.806122448979592}


In [ ]:
import pickle

model = dt

# Save the model to a .pkl file
with open('model_and_vectorizer.pkl', 'wb') as model_file:
    pickle.dump((model,vectorizer), model_file)

# To load the model back from the .pkl file
with open('model.pkl', 'rb') as model_file:
    loaded_model = pickle.load(model_file)
